In [51]:
import cv2
import os
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import numpy as np
from copy import copy
import math
import traceback
import matplotlib.pyplot as plt
import random
from skimage import transform

In [2]:
%matplotlib inline

In [3]:
def middle(cnt):
    M = cv2.moments(cnt)
    if M['m00'] == 0:
        return (0, 0)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    return (cx, cy)

def from_label(label_img):
    """Read one labeled image, and save the interested location.
    label_img (str), the path of a labeled image."""
    hsv = cv2.cvtColor(label_img, cv2.COLOR_BGR2HSV)
    lower_green = np.array([50,100, 50])
    upper_green = np.array([70,255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
    points = [middle(cnt) for cnt in cnts]
    return points

In [4]:
def islabel(point, cnt):
    return cv2.pointPolygonTest(cnt, point, True) >= -3

In [49]:
def watershed_labels(image):
# the lines is adapted from https://www.pyimagesearch.com/2015/11/02/watershed-opencv
    shifted = cv2.GaussianBlur(image, (7, 7), 0)
    gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    thresh = cv2.erode(thresh, np.ones((7, 7),np.uint8), iterations=2)
    
    D = ndimage.distance_transform_edt(thresh)
    localMax = peak_local_max(D, indices=False, min_distance=5, labels=thresh)
    markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
    labels = watershed(D * (-1), markers, mask=thresh)
    return labels, gray.shape

def segment(img, label, dst, name):
    """Segment possibel cells from a large image. Then, based on the labeled image, interested part 
    and non-interested part are divided into two groups. The small cell images are saved as .png files
    under the dst directory.
    file (str): the path of a primary image;
    lable (str): the path of a labeled image;
    dst (str): the path of the output directory."""
    print(img, label)
    image = cv2.resize(cv2.imread(img), (1920, 1440))
    label_img = cv2.resize(cv2.imread(label), (1920, 1440))
    points = from_label(label_img)
    
    mask_out = np.zeros(image.shape[:2], np.uint8)
    labels, gray_shape = watershed_labels(image)
                    
    for label in np.unique(labels):
        if label == 0:
            continue
        mask = np.zeros(gray_shape, dtype="uint8")
        mask[labels == label] = 255

        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
        c = max(cnts, key=cv2.contourArea)
        if any([islabel(point, c) for point in points]):
            cv2.drawContours(mask_out, [c], -1, 255, -1)
    
    cv2.imwrite(os.path.join(dst ,'{}.png'.format(name)), mask_out)
#     plt.figure(dpi=300)
#     plt.imshow(image)

In [6]:
#  https://agniva.me/scipy/2016/10/25/contour-smoothing.html
from scipy.interpolate import splprep, splev
def smooth(contour):
    x,y = contour.T
    # Convert from numpy arrays to normal arrays
    x = x.tolist()[0]
    y = y.tolist()[0]
    # https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.interpolate.splprep.html
    tck, u = splprep([x,y], u=None, s=1.0, per=1)
    # https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linspace.html
    u_new = np.linspace(u.min(), u.max(), 25)
    # https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.interpolate.splev.html
    x_new, y_new = splev(u_new, tck, der=0)
    # Convert it back to numpy format for opencv to be able to display it
    res_array = [[[int(i[0]), int(i[1])]] for i in zip(x_new,y_new)]
    return np.asarray(res_array, dtype=np.int32)

In [41]:
def couple(base, label):
    imgs = [os.path.join(base, f) for f in os.listdir(base)]
    labels = dict([(f.split('.')[0].replace('label', ''), os.path.join(label, f)) for f in os.listdir(label)])
    for img in imgs:
        name = os.path.basename(img).split('.')[0]
        yield img, labels[name], name

def main(base, label, dst):
    os.makedirs(dst, exist_ok=True)
    count, nocount = 0, 0
    for img, label, name in couple(base, label):
        try:
            segment(img, label, dst, name)
        except:
            print(traceback.format_exc())

In [50]:
main('../raw/', '.', '../mask')

../raw/2018-9-5_X007-40-1_ch00.tif .\2018-9-5_X007-40-1_ch00label.tif
../raw/2018-9-5_X009-2-40-1_ch00.tif .\2018-9-5_X009-2-40-1_ch00label.tif
../raw/EOS-340-1-40X.tif .\EOS-340-1-40Xlabel.tif
../raw/EOS-340-2-40X_A.tif .\EOS-340-2-40X_Alabel.tif
../raw/EOS-94-1-40X_A.tif .\EOS-94-1-40X_Alabel.tif
../raw/EOS-96-1-40X.tif .\EOS-96-1-40Xlabel.tif
../raw/EOS-96-2-40X.tif .\EOS-96-2-40Xlabel.tif


In [53]:
list(couple('../raw/', '../mask/'))

[('../raw/2018-9-5_X007-40-1_ch00.tif',
  '../mask/2018-9-5_X007-40-1_ch00.png',
  '2018-9-5_X007-40-1_ch00'),
 ('../raw/2018-9-5_X009-2-40-1_ch00.tif',
  '../mask/2018-9-5_X009-2-40-1_ch00.png',
  '2018-9-5_X009-2-40-1_ch00'),
 ('../raw/EOS-340-1-40X.tif', '../mask/EOS-340-1-40X.png', 'EOS-340-1-40X'),
 ('../raw/EOS-340-2-40X_A.tif',
  '../mask/EOS-340-2-40X_A.png',
  'EOS-340-2-40X_A'),
 ('../raw/EOS-94-1-40X_A.tif', '../mask/EOS-94-1-40X_A.png', 'EOS-94-1-40X_A'),
 ('../raw/EOS-96-1-40X.tif', '../mask/EOS-96-1-40X.png', 'EOS-96-1-40X'),
 ('../raw/EOS-96-2-40X.tif', '../mask/EOS-96-2-40X.png', 'EOS-96-2-40X')]

In [63]:
def data_aug(imgs, labels, angel=30, resize_rate=0.9):
    for img, label, name in couple(imgs, labels):
        image = cv2.imread(img)
        label = cv2.imread(label, 0)
        for batch in datagen.flow(image, batch_size=1, seed=42): 
            cv2.imwrite(os.path.join(imgs, 'aug_{}.tif'.format(name)), batch)
        for batch in datagen.flow(label, batch_size=1, seed=42):
            cv2.imwrite(os.path.join(labels, 'aug_{}.png'.format(name)), batch)

In [ ]:
# Create image generator
data_gen_args = dict(rotation_range=5,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     validation_split=0.2)
image_datagen = ImageDataGenerator(**data_gen_args)

seed = 1

def XYaugmentGenerator(X1, y, seed, batch_size):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=seed)
    genX2 = gen.flow(y, X1, batch_size=batch_size, seed=seed)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()

        yield X1i[0], X2i[0]


# Train model
model.fit_generator(XYaugmentGenerator(images, masks, seed, batch_size), steps_per_epoch=np.ceil(float(len(images)) / float(batch_size)),
                validation_data = XYaugmentGenerator(images_valid, masks_valid, batch_size), 
                validation_steps = np.ceil(float(len(images_valid)) / float(batch_size))
, shuffle=True, epochs=20)

In [64]:
data_aug('../raw/','../mask')